<center> 
    <h1> 
        Data Camp Project

 <center>
    <h1> 
        Scrapping Notebook

<div style="text-align: right">
    <em>
        See 'scrapping/scrapping_functions.py' file for the designed functions
    </em>
</div>

## 0. Installing & Importing dependencies

### Packages required 
* requests
* html5lib
* bs4
* bs4 selenium

In [2]:
# Import necessary packages
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from parse import search
from selenium import webdriver

In [3]:
# Set path to the chromedriver
driver_path = r"chromedriver.exe"

#Instagram account infos
user_name = "datacamp1020"
password= "$datacamp1020$"

In all this notebook we will use three functions of a class designed for this challenge. They are available and called from the file 'scraping_functions.py' (in the data folder of the repo, the same as this notebook is in)

In [ ]:
# Designed scrapping functions see 'scrapping_functions.py' 
from scrapping_functions import Scrapper
# Define our Scrapper class
scrapper = Scrapper(user_name, password, driver_path)

## 1. Retrieving top 49 World Influencers

We use a designed scrapping function to connect to this [Wikipedia web page](https://fr.m.wikipedia.org/wiki/Liste_des_comptes_Instagram_les_plus_suivis) to retrieve the 50 most followed instagram accounts (brands', institutions' or magazines' accounts will be deleted)

In [4]:
influencers_int, links_int = scrapper.get_influencers_world()

In [5]:
export_csv = influencers_int.to_csv (r'0_scrapping_data\influencer_list_world.csv', index = None, header=True) 

## 2. Retrieving influencers infos and their posts' links

We again use two designed functions (avalaible in the scrapping_functions.py file) to connect to instagram using the account informations. The function goes to each followers instagram page and scroll to retrieve a list of publications for each one.  

In [ ]:
#Can select only a few numbers of influencers to retrieve posts from to prevent connection problems or account ban 
#links_int_1 = links_int[0:10]
#influencers_int_1 = influencers_int[0:10]

In [ ]:
data_frames=[]
pause_time = 3
for i in range(len(links_int)):
    print(links_int[i])
    if i == 0:
        browser = scrapper.connection_instagram()
    df = scrapper.get_influencer_posts(influencers_int,i, browser, links_int[i], pause_time)
    data_frames.append(df)
links_pub_int = pd.concat(data_frames)
browser.quit()

In [ ]:
links_pub_int = pd.concat(data_frames)

In [ ]:
export_csv = links_pub_int.to_csv(r'0_scrapping_data\publications_list.csv', index = None, header=True) 

## 3. Retrieving posts' infos

We use the links scrapped right above to go to each one and scrap the related informations (use of the designed function get_info_pub of the class Scrapper in the file scrapping_functions.py).

In [46]:
publications = pd.read_csv(r'0_scrapping_data\publications_list.csv') 

In [ ]:
df_post = [] 
browser = scrapper.connection_instagram()
for pub_link in publications["Links to publication"]: 
    print(publications[publications["Links to publication"]==pub_link].index[0])
    df = scrapper.get_pub_info(pub_link,browser)
    df_post.append(df)
browser.quit()

In [ ]:
posts_info = pd.concat(df_post, ignore_index = True)
posts_info

In [93]:
df_2 = pd.merge(publications, post_info, how ='right', on ="Links to publication")

In [97]:
export_csv = df_2.to_csv (r'0_scrapping_data\raw_final_dataset.csv', index = None, header=True) 

## 4. Simple pre-processing

We use the merged dataset ollected from scrapping right above and process to a simple data pre-processing for convenience and prevent polluting the main notebook presenting the challenge

In [ ]:
raw_data = pd.read_csv(r'0_scrapping_data\raw_final_dataset.csv')

#### Deleting unecessary columns and renaming useful ones

In [ ]:
data_inst = raw_data.drop(columns = ['Links to publication', 'Links to profile', 'Account description',
                       'Verified Status'])
data_inst.columns = ['influencer','pr_activity','num_posts','num_followers','num_followings','post_description',
             'num_comments','num_likes','posting_date','media_type']

data_inst = data_inst.reset_index(drop=True)
print(data_inst.shape)
data_inst.head(3)

#### Cleaning the rows 

In [ ]:
def clean(row):
    return str(row).replace(",","").replace(" ","")

In [ ]:
data_inst['num_likes'] = data_inst['num_likes'].apply(clean).astype(int)  
data_inst['num_comments'] = data_inst['num_comments'].astype(int)
data_inst['num_followers'] = data_inst['num_followers'].apply(clean).astype(int)  
data_inst['num_followings'] = data_inst['num_followings'].apply(clean).astype(int) 
data_inst['num_posts'] = data_inst['num_posts'].apply(lambda x : x.replace("publications",""))
data_inst['num_posts'] = data_inst['num_posts'].apply(lambda x : x.replace("posts",""))
data_inst['num_posts'] = data_inst['num_posts'].apply(clean)

#### Re-organizing professional activities

In [ ]:
data_inst['pr_activity'] = data_inst['pr_activity'].apply(lambda x : x.lower())
data_inst['pr_activity'] = data_inst['pr_activity'].apply(lambda x : x.replace(' et ',''))
data_inst['pr_activity'] = data_inst['pr_activity'].apply(lambda x : x.replace('cienne','cien'))
data_inst['pr_activity'] = data_inst['pr_activity'].apply(lambda x : x.replace('trice','teur'))
data_inst['pr_activity'] = data_inst['pr_activity'].apply(lambda x : x.replace('comédienne','acteur'))
data_inst['pr_activity'] = data_inst['pr_activity'].replace({'footballeur':'Athlete',
                                        'joueur de basket-ball':'Athlete','acteur':'Actor',
                                        'acteurcatcheur professionnel':'Actor''musicienacteur':'Musician',
                                        'joueur de cricket':'Athlete','ex-footballeur':'Athlete',
                                        'acteurmusicien':'Actor','modèle':'Model','musicien':'Musician',
                                         'personnalité de téléréalitémodèle': 'TV Star',
                                         'personnalité de téléréalité':'TV Star'})

In [ ]:
data_inst['pr_activity'].unique()

#### Deleting brands and institutions

In [ ]:
data_inst = data_inst[data_inst['pr_activity']!='Marque']
data_inst = data_inst[data_inst['pr_activity']!='réseau social']
data_inst.shape

In [ ]:
data_inst['influencer'].unique()

#### Timing features extraction

In [ ]:
data_inst['posting_date'] = pd.to_datetime(data_inst["posting_date"])
data_inst['year'] = data_inst['posting_date'].dt.year
data_inst['month'] = data_inst['posting_date'].dt.month
data_inst['day'] = data_inst['posting_date'].dt.day
data_inst['hour'] = data_inst['posting_date'].dt.hour
data_inst['Day_week'] = data_inst['posting_date'].dt.day_name()
data_inst = data_inst.drop(columns = ['posting_date'])

In [ ]:
print(data_inst.shape)
data_inst.head(5)

## 5. Final Export

In [ ]:
export_csv = data_inst.to_csv (r'0_scrapping_data\data_pre_processed.csv', index = None, header=True) 